In [1]:
import json

from doml_mc.model.doml_model import parse_doml_model

In [2]:
from doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("assets/doml_meta.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [3]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [4]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')
pp.pprint(mm["application_SoftwarePackage"])

DOMLClass(
    name='application_SoftwarePackage',
    superclass='application_ApplicationComponent',
    attributes={
        'isPersistent': DOMLAttribute(
            name='isPersistent',
            type='Boolean',
            multiplicity=('1', '1'),
            default=False
        )
    },
    associations={
        'exposedInterfaces': DOMLAssociation(
            name='exposedInterfaces',
            class_='application_SoftwareInterface',
            multiplicity=('0', '*')
        ),
        'consumedInterfaces': DOMLAssociation(
            name='consumedInterfaces',
            class_='application_SoftwareInterface',
            multiplicity=('0', '*')
        )
    }
)


### Load the DOML document here:

In [5]:
doml_document_path = "example_json_models/wordpress_json_no_iface.doml"
with open(doml_document_path) as jsonf:
    doc = json.load(jsonf)

doml_model = parse_doml_model(doc, mm)


In [6]:
from doml_mc.intermediate_model.doml_model2im import doml_model_to_im
from doml_mc.intermediate_model.doml_element import reciprocate_inverse_associations
im = doml_model_to_im(doml_model, mm)
reciprocate_inverse_associations(im, inv_assoc)

In [7]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')
pp.pprint(im["wordpress"])

DOMLElement(
    name='wordpress',
    class_='application_SoftwarePackage',
    attributes={'commons_DOMLElement::name': 'wordpress'},
    associations={
        'application_SoftwarePackage::consumedInterfaces': {
            'postgres_db_interface'
        },
        'application_SoftwarePackage::exposedInterfaces': set()
    }
)


In [8]:
# del im["postgres_db_interface"].attributes["application_SoftwareInterface::endPoint"]

In [9]:
unbound_elems_n = 3
unbound_elems = [f"unbound{i}" for i in range(unbound_elems_n)]

In [10]:
from z3 import Solver

from doml_mc.z3.metamodel_encoding import (
    def_association_rel_and_assert_constraints,
    def_attribute_rel_and_assert_constraints,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from doml_mc.z3.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from doml_mc.z3.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm)
assoc_sort, assoc = mk_association_sort_dict(mm)
attr_sort, attr = mk_attribute_sort_dict(mm)
elem_sort, elem = mk_elem_sort_dict(im, unbound_elems)
ss_sort, ss = mk_stringsym_sort_dict(im, mm)
AData = mk_adata_sort(ss_sort)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
attr_rel = def_attribute_rel_and_assert_constraints(
    mm,
    solver,
    attr_sort,
    attr,
    class_,
    elem_class_f,
    elem_sort,
    AData,
    ss
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)
assoc_rel = def_association_rel_and_assert_constraints(
    mm,
    solver,
    assoc_sort,
    assoc,
    class_,
    elem_class_f,
    elem_sort,
    inv_assoc
)
assert_im_associations_q(
    assoc_rel,
    solver,
    {k: v for k, v in im.items() if k not in unbound_elems},
    elem,
    assoc_sort,
    assoc,
)

In [11]:
# solver.push()

### Metamodel statistics

In [12]:
from doml_mc.intermediate_model.metamodel import get_subclasses_dict

print("Number of classes:", len(mm))
print(
    "Number of tuples in the subclass relation:",
    sum(len(s) for s in get_subclasses_dict(mm).values())
)
n_attrs = sum(len(c.attributes) for c in mm.values())
print("Number of attributes:", n_attrs)
print("Number of attribute assertions:", 3 * n_attrs)
n_assocs = sum(len(c.associations) for c in mm.values())
print("Number of associations:", n_assocs)
print("Number of associations assertions:", 3 * n_assocs + len(inv_assoc))



Number of classes: 35
Number of tuples in the subclass relation: 92
Number of attributes: 24
Number of attribute assertions: 72
Number of associations: 46
Number of associations assertions: 142


### DOML model statistics

In [13]:
n_elems = len(im)
print("Number of elements:", n_elems)
n_attrs_im = sum(len(e.attributes) for e in im.values())
print("Number of attributes (IM):", n_attrs_im)
n_assocs_im = sum(len(a) for e in im.values() for a in e.associations.values())
print("Number of associations (IM):", n_assocs_im)
print("Number of string symbols:", len(ss))
print("Number of IM attribute assertions:", n_elems)
print("Number of IM association assertions:", n_elems ** 2)

Number of elements: 14
Number of attributes (IM): 18
Number of associations (IM): 18
Number of string symbols: 17
Number of IM attribute assertions: 14
Number of IM association assertions: 196


In [14]:
solver.check()

sat

In [15]:
solver.statistics()

(:added-eqs               24662
 :arith-make-feasible     5
 :arith-max-columns       7
 :binary-propagations     350
 :conflicts               89
 :datatype-accessor-ax    355
 :datatype-constructor-ax 9117
 :datatype-splits         1144
 :decisions               43772
 :del-clause              110
 :final-checks            6
 :max-generation          4
 :max-memory              24.34
 :memory                  24.30
 :mk-bool-var             2731
 :mk-clause               805
 :num-allocs              396711
 :num-checks              1
 :propagations            6672
 :quant-instantiations    553
 :restarts                2
 :rlimit-count            2609408
 :time                    0.49)

In [16]:
from z3 import Consts, ForAll, Exists, Implies, And, Or

In [17]:
# All software packages can see the interfaces they need through a common
# network.
spp, spc, i, n, ni, cn, c, d, dc = Consts(
    "spp spc i n ni cn c d dc", elem_sort
)
assn = ForAll(
    [spp, spc, i],
    Implies(
        And(
            assoc_rel(spp, assoc["application_SoftwarePackage::exposedInterfaces"], i),
            assoc_rel(spc, assoc["application_SoftwarePackage::consumedInterfaces"], i),
        ),
        Exists(
            [n],
            And(
                Or(
                    Exists(
                        [cn, d, ni],
                        And(
                            assoc_rel(d, assoc["commons_Deployment::source"], spp),
                            assoc_rel(d, assoc["commons_Deployment::target"], cn),
                            assoc_rel(cn, assoc["infrastructure_ComputingNode::ifaces"], ni),
                            assoc_rel(
                                ni, assoc["infrastructure_NetworkInterface::belongsTo"], n
                            ),
                        ),
                    ),
                    Exists(
                        [cn, d, c, dc, ni],
                        And(
                            assoc_rel(d, assoc["commons_Deployment::source"], spp),
                            assoc_rel(d, assoc["commons_Deployment::target"], c),
                            assoc_rel(dc, assoc["commons_Deployment::source"], c),
                            assoc_rel(dc, assoc["commons_Deployment::target"], cn),
                            assoc_rel(cn, assoc["infrastructure_ComputingNode::ifaces"], ni),
                            assoc_rel(
                                ni, assoc["infrastructure_NetworkInterface::belongsTo"], n
                            ),
                        ),
                    ),
                ),
                Or(
                    Exists(
                        [cn, d, ni],
                        And(
                            assoc_rel(d, assoc["commons_Deployment::source"], spc),
                            assoc_rel(d, assoc["commons_Deployment::target"], cn),
                            assoc_rel(cn, assoc["infrastructure_ComputingNode::ifaces"], ni),
                            assoc_rel(
                                ni, assoc["infrastructure_NetworkInterface::belongsTo"], n
                            ),
                        ),
                    ),
                    Exists(
                        [cn, d, c, dc, ni],
                        And(
                            assoc_rel(d, assoc["commons_Deployment::source"], spc),
                            assoc_rel(d, assoc["commons_Deployment::target"], c),
                            assoc_rel(dc, assoc["commons_Deployment::source"], c),
                            assoc_rel(dc, assoc["commons_Deployment::target"], cn),
                            assoc_rel(cn, assoc["infrastructure_ComputingNode::ifaces"], ni),
                            assoc_rel(
                                ni, assoc["infrastructure_NetworkInterface::belongsTo"], n
                            ),
                        ),
                    ),
                ),
            ),
        ),
    ),
)
solver.assert_and_track(assn, "software_package_iface_net")

e1, e2, ni = Consts("e1 e2 i", elem_sort)
assn = ForAll([e1, e2, ni],
    Implies(
        And(
            Or(
                assoc_rel(e1, assoc["infrastructure_ComputingNode::ifaces"], ni),
                assoc_rel(e1, assoc["infrastructure_Storage::ifaces"], ni),
            ),
            Or(
                assoc_rel(e2, assoc["infrastructure_ComputingNode::ifaces"], ni),
                assoc_rel(e2, assoc["infrastructure_Storage::ifaces"], ni),
            )
        ),
        e1 == e2
    )
)
solver.assert_and_track(assn, "iface_uniq")

In [18]:
solver.check()

sat

In [19]:
solver.statistics().memory

29.15

In [20]:
m = solver.model()

In [21]:
from itertools import product

In [22]:
for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()):
    if (e1n in unbound_elems or e2n in unbound_elems) and m.eval(assoc_rel(e1, a, e2)):
        print(e1, a, e2)

wpvm infrastructure_ComputingNode::ifaces unbound2
net1 infrastructure_Network::connectedIfaces unbound2
unbound0 commons_Deployment::source postgres
unbound0 commons_Deployment::target wpvm
unbound1 commons_Deployment::source wordpress
unbound1 commons_Deployment::target wpvm
unbound2 infrastructure_NetworkInterface::belongsTo net1
